In [99]:
import numpy as np 
import pandas as pd
import math
import os
from collections import namedtuple



In [123]:
print(os.listdir('data'))

['vrp_200_16_1', 'vrp_101_10_1', 'vrp_5_4_1', 'vrp_31_9_1', 'vrp_45_4_1', 'vrp_30_3_1', 'vrp_421_41_1', 'vrp_324_16_1', 'vrp_33_3_1', 'vrp_101_14_1', 'vrp_23_3_1', 'vrp_76_7_2', 'vrp_76_10_2', 'vrp_76_9_3', 'vrp_76_14_2', 'vrp_397_34_1', 'vrp_76_9_2', 'vrp_262_25_1', 'vrp_400_18_1', 'vrp_33_4_1', 'vrp_101_11_2', 'vrp_200_17_2', 'vrp_36_11_1', 'vrp_101_11_4', 'vrp_30_4_1', 'vrp_101_11_3', 'vrp_301_28_1', 'vrp_21_4_1', 'vrp_151_12_2', 'vrp_481_38_1', 'vrp_16_5_1', 'vrp_76_8_1', 'vrp_151_14_1', 'vrp_76_15_1', 'vrp_22_4_1', 'vrp_361_33_1', 'vrp_200_16_2', 'vrp_33_5_1', 'vrp_101_14_2', 'vrp_41_14_1', 'vrp_76_7_1', 'vrp_76_10_1', 'vrp_23_5_1', 'vrp_256_14_1', 'vrp_76_14_1', 'vrp_76_9_1', 'vrp_151_15_1', 'vrp_253_27_1', 'vrp_51_5_1', 'vrp_101_11_1', 'vrp_200_17_1', 'vrp_26_8_1', 'vrp_72_4_1', 'vrp_484_19_1', 'vrp_151_12_1', 'vrp_16_3_1', 'vrp_21_6_1', 'vrp_101_8_1', 'vrp_386_47_1', 'vrp_151_14_3', 'vrp_22_6_1', 'vrp_241_22_1', 'vrp_135_7_1', 'vrp_121_7_1', 'vrp_48_4_1', 'vrp_321_30_1', 'vrp_7

In [100]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [101]:
Customer = namedtuple("Customer", ['index', 'demand', 'x', 'y'])

In [102]:
def length(customer1, customer2):
    return math.sqrt((customer1.x - customer2.x)**2 + (customer1.y - customer2.y)**2)

In [103]:
def parse_data(input_data):
    # parse the input
    lines = input_data.split('\n')

    parts = lines[0].split()
    customer_count = int(parts[0])
    vehicle_count = int(parts[1])
    vehicle_capacity = int(parts[2])
    
    customers = []
    for i in range(1, customer_count+1):
        line = lines[i]
        parts = line.split()
        customers.append(Customer(i-1, int(parts[0]), float(parts[1]), float(parts[2])))

    #the depot is always the first customer in the input
    depot = customers[0] 
    return customer_count,vehicle_count,vehicle_capacity,customers,depot

In [104]:
def get_input_data(index):
    file_location = 'data/'+os.listdir('data')[index]
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    return input_data

In [135]:
customer_count,vehicle_count,vehicle_capacity,customers,depot  = parse_data(get_input_data(-13))

In [136]:
def get_distance_matrix(customer_count,customers):
    distance_matrix = np.zeros((customer_count,customer_count))
    for i in range(customer_count):
        for j in range(i,customer_count):
            d = length(customers[i],customers[j])
            distance_matrix[i,j] = d
            distance_matrix[j,i] = d
    return distance_matrix

In [137]:
def create_data_model(customer_count,vehicle_count,vehicle_capacity,customers):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = get_distance_matrix(customer_count,customers)
    data['num_vehicles'] = vehicle_count
    data['demands'] = [c.demand for c in customers]
    data['vehicle_capacities'] = [vehicle_capacity]*vehicle_count
    data['depot'] = 0
    return data

In [138]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [139]:
def main(customer_count,vehicle_count,vehicle_capacity,customers):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(customer_count,vehicle_count,vehicle_capacity,customers)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    #if solution:
    #    print_solution(data, manager, routing, solution)
    return routing,solution,manager

In [140]:
routing,solution,manager = main(customer_count,vehicle_count,vehicle_capacity,customers)

In [141]:
def get_obj_and_routes(solution,routing,manager,vehicle_count,customers):
    obj = solution.ObjectiveValue()
    vehicle_tours = [[] for i in range(vehicle_count)]
    for vehicle_id in range(vehicle_count):
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            vehicle_tours[vehicle_id].append(customers[node_index])
        vehicle_tours[vehicle_id].append(customers[manager.IndexToNode(index)])
    return obj,vehicle_tours

In [142]:
obj,vehicle_tours = get_obj_and_routes(solution,routing,manager,vehicle_count,customers)

In [143]:
obj

278

In [145]:
vehicle_tours

[[Customer(index=0, demand=0, x=30.0, y=40.0),
  Customer(index=2, demand=30, x=49.0, y=49.0),
  Customer(index=3, demand=16, x=52.0, y=64.0),
  Customer(index=8, demand=23, x=31.0, y=62.0),
  Customer(index=7, demand=19, x=17.0, y=63.0),
  Customer(index=0, demand=0, x=30.0, y=40.0)],
 [Customer(index=0, demand=0, x=30.0, y=40.0),
  Customer(index=1, demand=7, x=37.0, y=52.0),
  Customer(index=6, demand=15, x=21.0, y=47.0),
  Customer(index=14, demand=21, x=12.0, y=42.0),
  Customer(index=13, demand=23, x=5.0, y=25.0),
  Customer(index=4, demand=9, x=20.0, y=26.0),
  Customer(index=15, demand=10, x=36.0, y=16.0),
  Customer(index=0, demand=0, x=30.0, y=40.0)],
 [Customer(index=0, demand=0, x=30.0, y=40.0),
  Customer(index=11, demand=19, x=42.0, y=41.0),
  Customer(index=9, demand=11, x=52.0, y=33.0),
  Customer(index=10, demand=5, x=51.0, y=21.0),
  Customer(index=5, demand=21, x=40.0, y=30.0),
  Customer(index=12, demand=29, x=31.0, y=32.0),
  Customer(index=0, demand=0, x=30.0, y=4

In [ ]:
len(customers)